# Section 5 - Markov Models

## Basic definitions

>**Markov property**:
- The probability of a word occurring depends only on the previous word or a fixed number of preceding words, and not on the earlier words.

>**State transition matrix (denoted as `A`)**:
- A 2D matrix of the probabilities of transitioning from one state to another.

>**Initial state distribution (denoted as `π`)**:
- The probability distribution over the possible starting states.

>**Priors**:
- Probability distribution of the classes.

## Case Study - Building a Text Classifier

In [ ]:
# Download datasets
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt


NameError: name 'wget' is not defined